# Welcome to my Capstone project !

In [16]:
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup
import requests


In [17]:
url="https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M"
raw=requests.get(url).text
#Get Text Version of LXML code

soup = BeautifulSoup(raw,"lxml")
My_table = soup.find("table",{"class":"wikitable sortable"}) #obtain only table code and its contents from the Wikipedia Page
pcode=[]
borou=[]
neigh=[]

for row in My_table.findAll("tr"):
    cells = row.findAll("td")
    #For each "tr", assign each "td" to a variable.
    if len(cells)==3:
        pcode.append(cells[0].find(text=True))
        borou.append(cells[1].find(text=True))
        neigh.append(cells[2].find(text=True))
        
column=['Postcode','Borough','Neighborhood']
mainframe=pd.DataFrame(columns=column) #Generate empty dataframe

for i in range(0,len(pcode)): #Add all data into dataframe, except 'Not assigned' Boroughs
    if borou[i]!="Not assigned":
        mainframe.loc[i]=[pcode[i],borou[i],neigh[i]]

mainframe.reset_index(drop=True)
pcodes=mainframe['Postcode'].unique()     #Get list of postal codes

In [18]:
for i in pcodes:
    indexes=mainframe.loc[mainframe['Postcode']==i].index #Get list of indexes corresponding to each postcode
    text=mainframe['Neighborhood'][indexes[0]].replace('\n','')
    for j in range(0,len(indexes)):
        if j !=0:
            text_add=mainframe['Neighborhood'][indexes[j]].replace('\n','')
            text=text+", "+text_add #Prepare new text data for each unique Postcode
            mainframe.drop(indexes[j],inplace=True) #Drop rows except the merged row
    mainframe['Neighborhood'][indexes[0]]=text

In [19]:
mainframe.reset_index(drop=True,inplace=True)

In [20]:
NAindexes=mainframe.loc[mainframe['Neighborhood']=='Not assigned'].index #Get list of indexes corresponding to 'Not assigned' Neighborhood
for a in NAindexes:
    mainframe['Neighborhood'][a]=mainframe['Borough'][a]

In [21]:
#print("Number of rows in filtered dataframe: {}".format(mainframe.shape[0]))
#mainframe.head(11)

In [22]:
#Initialise new columns in the mainframe
mainframe['Latitude']=None 
mainframe['Longitude']=None
#Read CSV file
toadd=pd.DataFrame(pd.read_csv('http://cocl.us/Geospatial_data'))

In [23]:
#Correlate and add latitudes and longitudes from 'toadd' into mainframe
for i in pcodes:
    for j in toadd['Postal Code']:
        if i ==j:
            mainframe['Latitude'][mainframe.loc[mainframe['Postcode']==i].index]=toadd['Latitude'][toadd.loc[toadd['Postal Code']==j].index]
            mainframe['Longitude'][mainframe.loc[mainframe['Postcode']==i].index]=toadd['Longitude'][toadd.loc[toadd['Postal Code']==j].index]

In [24]:
mainframe.head(11)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.6543,-79.3606
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.7185,-79.4648
4,M7A,Queen's Park,Queen's Park,43.6623,-79.3895
5,M9A,Etobicoke,Islington Avenue,43.6679,-79.5322
6,M1B,Scarborough,"Rouge, Malvern",43.8067,-79.1944
7,M3B,North York,Don Mills North,43.7459,-79.3522
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.7064,-79.3099
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.6572,-79.3789
